In [0]:
from selenium import webdriver

In [0]:
!pip install selenium
!apt install chromium-chromedriver

In [0]:
!pip install -U -q Pydrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
url = 'https://drive.google.com/open?id=1dwaiV1_fIVBGHGeVKnRGXMgw-POve3I3'
id = url.split('=')[1]

In [86]:
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
wd.get("https://www.premierleague.com/players?se=79&cl=21")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: use options instead of chrome_options
  


---

* **선수명 불러오기**

In [0]:
player_list = wd.find_elements_by_class_name('playerName')

In [0]:
player_name = []

for i in range(0, len(player_list)):
    name = player_list[i].text
    player_name.append(name)
    
player_name

---

* **포지션 불러오기**

In [0]:
player_position_name = wd.find_elements_by_class_name('hide-s')

In [0]:
player_position = []

for i in range(2, len(player_position_name), 2):
    name = player_position_name[i].text
    player_position.append(name)

player_position

---

* **선수이름 + 포지션**

In [0]:
import pandas as pd

data = {'Name': player_name, 'Position': player_position}

team_sheet = pd.DataFrame(data)
team_sheet.head()

team_sheet = team_sheet.drop(49)

---

* **선수 링크 긁어오기**

In [0]:
test = wd.find_elements_by_tag_name('a')
test

In [0]:
link = []
for i in range(0, len(test)):
    a = test[i].get_attribute('href')
    link.append(a)
    
link

In [0]:
link = link[109:158]

In [0]:
link_number = []

for i in range(0, len(link)):
               a = int(link[i].split('/')[4])
               link_number.append(a)

            
# 선수 테이블과 합치기
team_sheet['id'] = link_number

---

## **지난 경기 기록 긁어오기**

In [0]:
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
match_wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
match_wd.get("https://www.premierleague.com/clubs/21/Tottenham-Hotspur/results?co=1&se=79")

In [0]:
# match_tag = match_wd.find_elements_by_tag_name('li')

match_num_01 = []

for i in range(1, 39):
    a = i * 3
    number = '#mainContent > div:nth-child(6) > div > div.wrapper.col-12.active > div.col-12 > section > div:nth-child(%d) > ul > li > div' % a
    b = match_wd.find_element_by_css_selector(number)
    
    match_num_01.append(b)

In [0]:
match_num_02 = []

for i in range(0, len(match_num_01)):
    a = match_num_01[i].get_attribute('data-matchid')
    match_num_02.append(a)
    
match_num_02 = list(map(int,match_num_02))
match_num_02

In [0]:
for i in match_num_02:
    a = 'https://www.premierleague.com/match/%d' % i
    print(a)

---

## **경기별 선수 명단 긁어오기**

In [0]:
# https://www.premierleague.com/match/22720
from bs4 import BeautifulSoup
import urllib.request
import urllib.parse

# web_url에 원하는 웹의 URL을 넣어주시면 됩니다.
web_url = "https://www.premierleague.com/match/22720"

with urllib.request.urlopen(web_url) as response:
    html = response.read()
    soup = BeautifulSoup(html, 'html.parser')    

players = soup.find_all('div', 'matchLineupTeamContainer')

In [0]:
len(players)

In [0]:
a.find_all("div", class_="name").t

In [651]:
a = players[i].find_all("div", class_="name")
a[0].get_text()

' Hugo Lloris '

In [697]:
player_name = []
for i in range(0, len(players)) :
    if (players[i].get_attribute_list('data-team-id')[0] == '21') :
        a = players[i].find_all("div", class_="name")
       # print(a)
        for j in range(0, len(a)) :
            player_name.append(a[j].get_text())

player_name

[' Hugo Lloris ',
 ' Eric Dier ',
 ' Kyle Walker-Peters ',
 ' Danny Rose ',
 ' Toby Alderweireld ',
 ' Victor Wanyama ',
 " Erik Lamela 78'",
 " Lucas Moura 74'",
 " Moussa Sissoko 84'",
 ' Christian Eriksen ',
 ' Harry Kane ',
 ' Michel Vorm ',
 ' Ben Davies ',
 ' Juan Foyth ',
 " Davinson Sánchez 78'",
 " Dele Alli 74'",
 " Son Heung-Min 84'",
 ' Fernando Llorente ']

In [699]:
lstrip_player_name = []

for i in range(0, len(player_name)):
    a = player_name[i].lstrip()
    lstrip_player_name.append(a)
    
lstrip_player_name

['Hugo Lloris ',
 'Eric Dier ',
 'Kyle Walker-Peters ',
 'Danny Rose ',
 'Toby Alderweireld ',
 'Victor Wanyama ',
 "Erik Lamela 78'",
 "Lucas Moura 74'",
 "Moussa Sissoko 84'",
 'Christian Eriksen ',
 'Harry Kane ',
 'Michel Vorm ',
 'Ben Davies ',
 'Juan Foyth ',
 "Davinson Sánchez 78'",
 "Dele Alli 74'",
 "Son Heung-Min 84'",
 'Fernando Llorente ']

* **정규식**
    * 출처1: http://melonicedlatte.com/python/2017/02/23/152309.html
    * 출처2: https://niceman.tistory.com/156

In [713]:
re_player_name = []
fin_player_name = []

import re

for i in range(0, len(lstrip_player_name)):
    text = lstrip_player_name[i]
    parse = re.sub('[' '][0-9][0-9][-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '', text)
    re_player_name.append(parse)

for j in range(0, len(re_player_name)):
    a = re_player_name[j].rstrip()
    fin_player_name.append(a)
        
fin_player_name

#     for j in range(0, len(re_player_name)):
#         a = re_player_name[j].rstrip()
#         fin_player_name.append(a)

['Hugo Lloris',
 'Eric Dier',
 'Kyle Walker-Peters',
 'Danny Rose',
 'Toby Alderweireld',
 'Victor Wanyama',
 'Erik Lamela',
 'Lucas Moura',
 'Moussa Sissoko',
 'Christian Eriksen',
 'Harry Kane',
 'Michel Vorm',
 'Ben Davies',
 'Juan Foyth',
 'Davinson Sánchez',
 'Dele Alli',
 'Son Heung-Min',
 'Fernando Llorente']